In [ ]:
%load_ext autoreload
%autoreload 2
from pybaseball import statcast, pitching_stats
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
from collections import Counter
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from matplotlib import patches
%matplotlib inline

# use Statcast data (from 2015-2018) so we can get spin rate, etc.
train_data_dates = [('2015-04-05', '2015-10-04'),      # 2015 data
                    ('2016-04-03', '2016-10-02'),       # 2016 data
                    ('2017-04-02', '2017-10-01'),       # 2017 data
                    ('2018-03-29', '2018-10-01')]       # 2018 data

### Get the Outcome Style Data (Groundball/flyball rates, strike percentages, whiff rates, etc.)

In [ ]:
# scrape the data from baseball savant
#pitcher_season_stats = pitching_stats(2018)
pitcher_season_stats = pd.read_csv("/Users/chrisjackson/sports/baseball/data/overall_data_2018.csv")

# select the features that we want to use to categorize the pitchers... questions we want to answer:
#   - are they strike throwers?
#   - do they allow a lot of baserunners? 
#   - groundball or flyball pitcher?
#   - are they swing and miss guys (inside and outside of the zone)?
cols_to_keep = ['Name', 'Strikes', 'Pitches', 
                'K/9', 'BB/9',  'H/9', 'HR/9',
                'GB/FB', 'LD%', 'GB%', 
                'O-Swing%', 'Z-Swing%', 'Swing%', 
                'O-Contact%', 'Z-Contact%', 'Contact%', 
                'Zone%', 'F-Strike%', 'SwStr%']
pitcher_season_stats = pitcher_season_stats[cols_to_keep]

# keep only pitchers with at least 500 pitches in a season... try to weed out the noise, but keep short relievers
pitcher_season_stats = pitcher_season_stats[pitcher_season_stats['Pitches'] >= 500]

# compute the percentage of pitches that are strikes
pitcher_season_stats['Strike Pct'] = pitcher_season_stats['Strikes'] / pitcher_season_stats['Pitches']
pitcher_season_stats.drop(['Strikes', 'Pitches'], axis=1, inplace=True)

# use the GB/FB ratio to compute the OF flyball pct
pitcher_season_stats['Flyball%'] = (pitcher_season_stats['GB%'] / pitcher_season_stats['GB/FB'])

# drop the GB/FB ratio
pitcher_season_stats.drop(['GB/FB'], axis=1, inplace=True)

# rename the Name column so we can merge on it later... and the hit ball types to make them more readable
pitcher_season_stats.rename(columns={'Name': 'player_name',
                                     'GB%': 'Groundball%',
                                     'LD%': 'Linedrive%'}, inplace=True)

# reorder the columns
pitcher_season_stats = pitcher_season_stats[[
    'player_name', 'Strike Pct', 'K/9', 'BB/9', 'H/9', 'HR/9', 
    'Linedrive%', 'Groundball%', 'Flyball%', 
    'O-Swing%', 'Z-Swing%', 'Swing%', 
    'O-Contact%', 'Z-Contact%', 'Contact%', 
    'Zone%', 'F-Strike%', 'SwStr%', 
       ]]

print(f"Number of features: {pitcher_season_stats.shape[1]-1}")
pitcher_season_stats.head()

### Get the Descriptive Statistics for Each Column

In [ ]:
pitcher_season_stats.describe()

### Get the Pitch-by-Pitch Data

In [ ]:
# get the data from baseball savant
#pitch_by_pitch_data = statcast(start_dt='2018-03-29', end_dt='2018-10-01')
pitch_by_pitch_data = pd.read_csv("/Users/chrisjackson/sports/baseball/data/pitch_data_2018.csv")

# choose the columns we want to use for identifying Families
cols_to_keep = ['pitcher', 'player_name', 'p_throws', 'pitch_type', 
                'release_speed', 'release_pos_x', 'release_pos_z', 'release_spin_rate', 
                'pfx_x', 'pfx_z'] #, 'plate_x', 'plate_z']
pitch_by_pitch_data = pitch_by_pitch_data[cols_to_keep]

# make sure pitcher ID's are ints
pitch_by_pitch_data['pitcher'] = pitch_by_pitch_data['pitcher'].astype(int)

pitch_by_pitch_data.head()

### Categorize Pitches as Fastball, Breaking Pitch or Off-Speed Pitch 

In [ ]:
# group the various pitch types into three types (fastball=FB, breaking=BR and off-speed=OS)
fastball_pitches = ['FC', 'FF', 'FA', 'FT', 'SI']
breaking_pitches = ['CU', 'KC', 'KN', 'SC', 'SL', 'GY']
offspeed_pitches = ['CH', 'EP', 'FO', 'FS']

# pitches to drop (pitch outs, intentional balls, uncategorized)
dropped_pitches = ['AB', 'AS', 'IN', 'NP', 'PO', 'UN']

# categorize pitches
def categorize_pitches(x):
    if x in fastball_pitches:
        return 'FB'
    elif x in breaking_pitches:
        return 'BR'
    elif x in offspeed_pitches:
        return 'OS'
    else:
        return x
pitch_by_pitch_data['pitch_type'] = pitch_by_pitch_data['pitch_type'].apply(categorize_pitches)

# drop any rows with non-pitch pitches
pitch_by_pitch_data = pitch_by_pitch_data[~pitch_by_pitch_data['pitch_type'].isin(dropped_pitches)]

# drop any rows with NaN for pitch type
pitch_by_pitch_data = pitch_by_pitch_data[pd.notnull(pitch_by_pitch_data['pitch_type'])]

# print out the breakdown of categorized pitch types
pitch_counts = Counter(pitch_by_pitch_data['pitch_type'])
print(pitch_counts)
print()
pitch_by_pitch_data.head()

### Compute Average Speed, Movement, Spin Rate and Location for Each Pitch Type (FB, BR and OS)

In [ ]:
# build a list of pitcher IDs
pitcher_list = list(set(pitch_by_pitch_data['pitcher'].tolist()))
print(f"{len(pitcher_list)} pitchers in the data.")

# initiate a dataframe to hold the data
pitcher_pitch_avgs_df = pd.DataFrame()

# loop thru pitchers in the list and construct their personal DF
for pitcher in pitcher_list:
    
    # select pitches thrown by pitcher
    pitcher_df = pitch_by_pitch_data[pitch_by_pitch_data['pitcher'] == pitcher]
    total_num_pitches = len(pitcher_df)

    # loop thru the pitch types
    pitch_types = ['FB', 'BR', 'OS']
    pitch_type_avgs = []
    for pitch_type in pitch_types:

        # select the pitches thrown by pitcher by type
        pitcher_pitch_df = pitcher_df[pitcher_df['pitch_type'] == pitch_type]
        num_pitches = len(pitcher_pitch_df)

        # groupby player name/throws/pitch_type and compute mean of remaining features
        pitcher_pitch_df = pd.DataFrame(pitcher_pitch_df.groupby(['pitcher', 'player_name', 'p_throws', 'pitch_type']).mean())
        
        # rename columns to have pitch type in name
        pitcher_pitch_df.columns = [pitch_type + ' ' + x for x in pitcher_pitch_df.columns.tolist()]

        pitcher_pitch_df.reset_index(inplace=True, drop=False)
        
        # compute the percentage of pitches thrown that are of pitch_type
        pitcher_pitch_df[pitch_type+'%'] = num_pitches / total_num_pitches

        pitcher_pitch_df.drop('pitch_type', axis=1, inplace=True)

        pitcher_pitch_df.set_index(['pitcher', 'player_name', 'p_throws'], inplace=True)

        # append pitch type dataframe to pitcher's dataframe
        pitch_type_avgs.append(pitcher_pitch_df)

    # append pitcher's dataframe to all-pitchers dataframe
    pitcher_pitch_avgs_df = pitcher_pitch_avgs_df.append(pd.concat(pitch_type_avgs, axis=1, join='inner'))

pitcher_pitch_avgs_df.reset_index(inplace=True, drop=False)

pitcher_pitch_avgs_df.head()

### convert release position x, z coordinates into an "arm slot angle" (measured from vertical)

In [ ]:
pitcher_pitch_avgs_df['FB arm_angle'] = np.abs((180 / 3.14) * np.arctan(pitcher_pitch_avgs_df['FB release_pos_x'] / pitcher_pitch_avgs_df['FB release_pos_z']))
pitcher_pitch_avgs_df['BR arm_angle'] = np.abs((180 / 3.14) * np.arctan(pitcher_pitch_avgs_df['BR release_pos_x'] / pitcher_pitch_avgs_df['BR release_pos_z']))
pitcher_pitch_avgs_df['OS arm_angle'] = np.abs((180 / 3.14) * np.arctan(pitcher_pitch_avgs_df['OS release_pos_x'] / pitcher_pitch_avgs_df['OS release_pos_z']))
pitcher_pitch_avgs_df.drop(['FB release_pos_x', 'FB release_pos_z', 'BR release_pos_x', 'BR release_pos_z', 'OS release_pos_x', 'OS release_pos_z'], axis=1, inplace=True)

pitcher_pitch_avgs_df.head()

### Compute the Percent Difference Between OS and FB Speeds

In [ ]:
pitcher_pitch_avgs_df['OS_FB_Diff'] = (pitcher_pitch_avgs_df['OS release_speed'] - pitcher_pitch_avgs_df['FB release_speed']) / (pitcher_pitch_avgs_df['FB release_speed'])

pitcher_pitch_avgs_df.head()

### Merge the Outcome Style Data with the Pitch Characteristic Data

In [ ]:
pitcher_data = pd.merge(pitcher_pitch_avgs_df, pitcher_season_stats, how='inner', on='player_name')

print(f"Shape of the final training data: {pitcher_data.shape}")
pitcher_data.head()

### Split Into Two Groups: Lefties and Righties

In [ ]:
lh_pitcher_data = pitcher_data[pitcher_data['p_throws'] == 'L']
print(f"Number of LH pitchers: {lh_pitcher_data.shape[0]}")

rh_pitcher_data = pitcher_data[pitcher_data['p_throws'] == 'R']
print(f"Number of RH pitchers: {rh_pitcher_data.shape[0]}")

## Rescaling, PCA and Clustering 

### Helper functions: Rescale the Data, Perform PCA and Find Number of Dimensions to Explain 98% of the Variance, Plot the clusters, etc.

In [ ]:
# custom scatter plot function which allows for markers to take a list
def mscatter(x,y, ax=None, m=None, **kw):
    import matplotlib.markers as mmarkers
    ax = ax or plt.gca()
    sc = ax.scatter(x,y,**kw)
    if (m is not None) and (len(m)==len(x)):
        paths = []
        for marker in m:
            if isinstance(marker, mmarkers.MarkerStyle):
                marker_obj = marker
            else:
                marker_obj = mmarkers.MarkerStyle(marker)
            path = marker_obj.get_path().transformed(
                        marker_obj.get_transform())
            paths.append(path)
        sc.set_paths(paths)
    return sc

# perform k-means clustering and use the Gap Statistic to find optimal k
def optimalK(data, nrefs=3, maxClusters=10):
    """
    Calculates KMeans optimal K using Gap Statistic from Tibshirani, Walther, Hastie
    Params:
        data: ndarry of shape (n_samples, n_features)
        nrefs: number of sample reference datasets to create
        maxClusters: Maximum number of clusters to test for
    Returns: (gaps, optimalK)
    """
    gaps = np.zeros((len(range(1, maxClusters)),))
    resultsdf = pd.DataFrame({'clusterCount':[], 'gap':[]})
    for gap_index, k in enumerate(range(1, maxClusters)):

        # Holder for reference dispersion results
        refDisps = np.zeros(nrefs)

        # For n references, generate random sample and perform kmeans getting resulting dispersion of each loop
        for i in range(nrefs):

            # Create new random reference set
            randomReference = np.random.random_sample(size=data.shape)

            # Fit to it
            km = KMeans(k, n_jobs=-1)
            km.fit(randomReference)

            refDisp = km.inertia_
            refDisps[i] = refDisp

        # Fit cluster to original data and create dispersion
        km = KMeans(k)
        km.fit(data)

        origDisp = km.inertia_

        # Calculate gap statistic
        gap = np.log(np.mean(refDisps)) - np.log(origDisp)

        # Assign this loop's gap statistic to gaps
        gaps[gap_index] = gap

        resultsdf = resultsdf.append({'clusterCount':k, 'gap':gap}, ignore_index=True)

    return (gaps.argmax() + 1, resultsdf)  # Plus 1 because index of 0 means 1 cluster is optimal, index 2 = 3 clusters are optimal



def perform_pca_clustering(data):

    data.reset_index(inplace=True, drop=True)

    # keep pitcher ID and name
    data_ID_name = data[['pitcher', 'player_name']]

    # data for clustering
    data_clustering = data.drop(['pitcher', 'player_name', 'p_throws'], axis=1)
    print(f"Number of original dimensions: {data_clustering.shape[1]}")

    # rescale 
    scaler = MinMaxScaler()
    data_clustering_scaled = pd.DataFrame(scaler.fit_transform(data_clustering))

    # find the minimal number of dimensions that captures 98% of the variance
    # fitting the PCA algorithm with our Data
    pca = PCA().fit(data_clustering_scaled)

    # the cumulative explained variance
    exp_var_ratio = list(np.cumsum(pca.explained_variance_ratio_))

    # use the number of dimensions that get us to 98% explained variance
    exp_var_threshold = 0.98
    for i, ev in enumerate(exp_var_ratio):
        if ev > exp_var_threshold:
            num_components = i
            break
        else:
            num_components = len(data_clustering_scaled.columns)

    print(f"{num_components} are required to explain 98% of the variance.")
    
    # perform PCA with computed number of dimensions
    pca = PCA(n_components=num_components)
    data_clustering_pca = pd.DataFrame(pca.fit_transform(data_clustering_scaled))

    # find the optimal k
    k, _ = optimalK(data_clustering_pca)
    print(f"Optimal k: {k}")

    # perform K-Means clustering with computed number of clusters
    kmeans = KMeans(n_clusters=k)
    kmeans.fit_transform(data_clustering_pca)
    labels = pd.DataFrame(kmeans.labels_)
    labels.rename(columns={0: 'Cluster_Number'}, inplace=True)

    # put the original data back together with a new column for the cluster number label
    pitchers_clusters = pd.concat([data_ID_name, labels, data_clustering], axis=1, join='inner')
    
    # re-do the analysis for visualization purposes
    # perform PCA with only 2dimensions
    pca = PCA(n_components=2)
    data_clustering_pca = pd.DataFrame(pca.fit_transform(data_clustering_scaled))

    kmeans = KMeans(n_clusters=k).fit(data_clustering_pca)

    labels = kmeans.labels_
    # map cluster number to marker 
    marker_map = {0: 'o', 1: 'v', 2: '^', 3: '>', 4: '<', 5: '8', 6: 's', 7: 'p',
                  8: 'P', 9: '*', 10: 'D', 11: 'X', 12: 'h', 13: 'H'}
    labels = [marker_map[x] for x in labels]

    data_clustering_pca['cluster_number'] = kmeans.predict(data_clustering_pca)

    data_clustering_pca['cluster_number'] = data_clustering_pca['cluster_number'].apply(lambda x: marker_map[x])
    
    data_cluster_w_name = pd.merge(data_ID_name, data_clustering_pca, left_index=True, right_index=True)
    print(data_cluster_w_name.head())
        
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111)
    ax2 = mscatter(data_clustering_pca.iloc[:, 0], data_clustering_pca.iloc[:, 1], ax=ax, m=labels, c=kmeans.labels_, s=80)
#    ax2 = plt.scatter(data_clustering_pca.iloc[:, 0], data_clustering_pca.iloc[:, 1], c=kmeans.labels_, s=80)
#    plt.legend(loc='best')
    
    return k, pitchers_clusters, ax2

### Analysis for Lefties

In [ ]:
lh_k, lh_pitchers_clusters, ax = perform_pca_clustering(lh_pitcher_data)
plt.title(f"Left-Handed Pitchers K-Means Clustering (k = {lh_k})")
plt.show()

### Analysis for Righties 

In [ ]:
rh_k, rh_pitchers_clusters, ax = perform_pca_clustering(rh_pitcher_data)
plt.title(f"Right-Handed Pitchers K-Means Clustering (k = {rh_k})")
plt.show()

## Qualitative Analysis of the Clusters

### Left-Handed Pitchers

In [ ]:
def get_group_box_plot(df, k, feat_name):
    feat_list = []
    for i in range(k):
        k_df = df[df['Cluster_Number'] == i]
        feat = k_df[feat_name].tolist()
        feat_list.append(feat)
    feat_avg = df[feat_name].mean()
    feat_std = df[feat_name].std()

    plt.boxplot(feat_list)
    plt.plot([1, k+1], [feat_avg+feat_std, feat_avg+feat_std], 'k--')
    plt.plot([1, k+1], [feat_avg, feat_avg], 'k--')
    plt.plot([1, k+1], [feat_avg-feat_std, feat_avg-feat_std], 'k--')
    plt.title(feat_name)
    plt.xlabel('Cluster Number')
    plt.show()

#### Feature Distributions for Each Cluster

The horizontal dashed lines represent the mean for all left-handers (and the 1 sigma bounds)

In [ ]:
for col in lh_pitchers_clusters.columns.tolist()[3:]:
    get_group_box_plot(lh_pitchers_clusters, lh_k, col)

#### Looks like Cluster 12 is Made up of Hard-Throwing Swing & Miss guys

In [ ]:
lh_pitchers_clusters[lh_pitchers_clusters['Cluster_Number'] == 11]

### Right-Handed Pitchers

In [ ]:
for col in rh_pitchers_clusters.columns.tolist()[3:]:
    get_group_box_plot(rh_pitchers_clusters, rh_k, col)

In [ ]:
rh_pitchers_clusters[rh_pitchers_clusters['Cluster_Number'] == 10]